In [1]:
from single_HMM import *
class HMM_train:
    def __init__(self):
        self.states: List[HMMState] = []
        self.hmms=List[HMM] = []
        self.observations: List[np.ndarray] = []
        self.transitions={}
        self.state_index: Dict[HMMState, int] = {}
        self.initial_probabilities: List[float] = [] 
        self.set_state_index()
    def add_digit(self,digit):
        if digit in range(0,9):
            filename='Digit {digit} HMM'
            hmm=load_hmm(filename)
            self.hmms.append(hmm)
        else:
            filename='Silence HMM'
            hmm=load_hmm(filename)
            self.hmms.append(hmm)
    def add_all_digit(self,sequence):#sequence is a 1 d list representing the recorded digit sequence e.g"187492"
        self.add_digit('Silence')
        for i in sequence:
            self.add_digit(i)
        self.add_digit('Silence')
    
    def construct_language_hmm_from_single_hmms(self):
        for hmm in self.hmms:
            for state in hmm.states:
                self.states.append(state)
        self.initialize_transitions_for_all_states()
        
    
    def initialize_transitions_for_all_states(self):
        #for the first hmm
        hmm=self.hmms[0]
        #for all states except for the first state
        for j in range(1,len(hmm.states)):
            hmm.states[j].parents.append(hmm.states[j-1])
            hmm.states[j].parents.append(hmm.states[j])
        #for the first state, the parent is only itself
        hmm.states[0].parents.append(hmm.states[0])
        #for all hmms except for the last hmm
        for i in range(len(self.hmms)-1):
            hmm=self.hmms[i]
            #for all states except for the first state
            for j in range(1,len(hmm.states)):
                hmm.states[j].parents.append(hmm.states[j-1])
                hmm.states[j].parents.append(hmm.states[j])
            #for the first state
                hmm.states[0].parents
            

In [2]:
def txt_to_int(txt):
    for i in range(10):
        if txt==f'{i}':
            return i

In [3]:
class unrestricted_recognizer:
    def __init__(self,hmms):
        self.hmms=hmms
        self.states=[]
        
        self.construct_states_from_hmms()
        

        self.initial_probability=1/len(self.hmms)
        self.transitions={}
        self.initialize_transitions()
    #suppose the first state in list self.hmms is a null state
    def construct_states_from_hmms(self):
        #add a null state in the beginning
        ns=HMMState(label1="Null State",isNull=True)
        self.states.append(ns)
        for hmm in self.hmms:
            for i in range(len(hmm.states)-1):
                state=hmm.states[i]
                full_label=state.label1
                
                state_label=full_label.split("-")[1]
                state_label_integer=txt_to_int(state_label)
                if state_label_integer==1:
                    state.parents.append(state)
                    state.parents.append(self.states[0])#null state
                else:
                    state.parents.append(hmm.states[state_label_integer-2])
                    state.parents.append(state)
                self.states.append(state)
        for hmm in self.hmms:
            self.states[0].parents.append(hmm.states[-1])
    def initialize_transitions(self):
        for state in self.states:
            self.transitions[state]={}
        for state in self.states:
            #if the state is null state
            if state.isNull==True:
                for parent in state.parents:
                    self.transitions[parent][state]=0.5
            #else
            else:
                for parent in state.parents:
                    #if the state's parent is null state
                    if parent.isNull==True:
                        self.transitions[parent][state]=1/len(self.hmms)
                    else:
                        #if the state and its parent are all emitting states, find the hmm model they belongs to
                        hmm=self.recognize_the_hmm_the_state_belongs_to_by_label(parent)
                        state_idx=self.recognize_the_index_of_the_state_in_digit_hmm_by_label(state)
                        parent_idx=self.recognize_the_index_of_the_state_in_digit_hmm_by_label(parent)
                        #if the state is the last state, and the parent is itsef
                        if state_idx==len(hmm.states)-1:
                            self.transitions[parent][state]=0.5
                            
                            
                        else:
                            
                            self.transitions[parent][state]=hmm.transitions[parent_idx][state_idx]
    



    def recognize_the_hmm_the_state_belongs_to_by_label(self,state):
        full_label=state.label1
        digit_label=full_label.split("-")[0]
        state_label_integer=txt_to_int(digit_label)
        hmm=self.hmms[state_label_integer]
        return hmm
    def recognize_the_digit_the_state_belongs_to_by_label(self,state):
        full_label=state.label1
        digit_label=full_label.split("-")[0]
        digit=txt_to_int(digit_label)
        return digit
    def recognize_the_index_of_the_state_in_digit_hmm_by_label(self,state):
        full_label=state.label1
        state_index=full_label.split("-")[1]
        state_label_integer=txt_to_int(state_index)
        return state_label_integer-1



    def get_parent_indices(self,child_state):
        
        parent_states=child_state.parents
        parent_indices=[]
        for p in parent_states:
            label=p.label1
            digit=txt_to_int(label.split("-")[0])
            rank=txt_to_int(label.split("-")[1])
            idx=5*digit+rank
            parent_indices.append(idx)
        return parent_indices
    def print_best_path(best_path):
        str=""
        for state in best_path:
            label=state.label1
            digit=label.split("-")[0]
            str.append(digit)
        print(f'The best path is: {str}')
    def get_first_state_idx_in_each_single_digit_hmm_in_reconizer_hmm(self):
        indices=[]
        num_first_states=len(self.hmms)
        for i in range(num_first_states):
            idx=len(self.hmms[0].states)*i+1
            indices.append(idx)
        return indices
            

    def alignment(self,seq):
        t_max=len(seq)
        num_states=len(self.states)
        Score=np.zeros((num_states,t_max))
        path = np.empty((num_states, t_max), dtype=object)
        first_state_indices=self.get_first_state_idx_in_each_single_digit_hmm_in_reconizer_hmm()
        #for the first column in DP score table score
        for i,row in enumerate(Score):
            #can only start at the null state
            if self.states[i].isNull==False:
                if i in first_state_indices:
                    Score[i][0]=math.log(self.initial_probability)
                else:
                    Score[i][0]=-math.inf
            else:
                Score[i][0]=0
    
        #for the first column in DP path table score
        for i,row in enumerate(path):
            #can only start at the null state
            path[i][0]=np.array([])
        for j in range(1,t_max):
            for i,row in enumerate(Score): #i = the idx of the current node
                path[i][j]=[]
                if self.states[i].isNull==False:
                    parents=self.states[i].parents
                    parent_indices=self.get_parent_indices(self.states[i])
                    max=-math.inf
                    best_parent=None
                    best_idx=None

                    for idx in parent_indices:
                        transition_p=self.transitions[idx][i]
                        transition_score=math.log(transition_p)
                        emission_score=self.states[i].log_multivariate_gaussian_pdf_diag_cov(seq[j])
                        total_score=Score[idx][j-1]+transition_score+emission_score
                        if total_score>max:
                            max=total_score
                            best_parent=parents[idx]
                            best_idx=idx
                    Score[i][j]=max
                    path[i][j]=np.concatenate((path[best_idx][j-1],np.array([best_parent])))
                else:
                    for idx in parent_indices:
                        transition_p=self.transitions[idx][i]
                        transition_score=math.log(transition_p)
                        total_score=Score[idx][j-1]+transition_score
                        if total_score>max:
                            max=total_score
                            best_parent=parents[idx]
                            best_idx=idx
                    Score[i][j]=max
                    path[i][j]=np.concatenate((path[best_idx][j-1],np.array([best_parent])))
        #for the last timestep
        max_idx=0
        max_score=-math.inf
        for i in range(num_states):
            if Score[i][t_max-1]>max_idx:
                max_idx=i
                max_score=Score[i][t_max-1]
        best_path=path[max_idx][t_max-1]
        self.print_best_path(best_path)
        return max_score,best_path
    def recognize(self,seq):
        max_score,best_path=self.alignment(seq)
        digit_seq=''
        prev_digit=-1
        for state in best_path:
            digit=self.recognize_the_digit_the_state_belongs_to_by_label(state)
            if digit!=prev_digit:
                digit_seq=digit_seq+str(digit)
                prev_digit=digit
        print(f'The number is recognized as {digit_seq}')
            

            

def print_transition_information(recognizer,i):
    key, value = list(recognizer.transitions.items())[i]
    f=key.label1
    print(f'Transit From {key}')
    print(f'Transit to {value}')
    print(f'The State Label is:{f}')





In [4]:
hmmss=load_all_hmm()


HMM model has been loaded from 'Digit 0 HMM'
HMM model has been loaded from 'Digit 1 HMM'
HMM model has been loaded from 'Digit 2 HMM'
HMM model has been loaded from 'Digit 3 HMM'
HMM model has been loaded from 'Digit 4 HMM'
HMM model has been loaded from 'Digit 5 HMM'
HMM model has been loaded from 'Digit 6 HMM'
HMM model has been loaded from 'Digit 7 HMM'
HMM model has been loaded from 'Digit 8 HMM'
HMM model has been loaded from 'Digit 9 HMM'


In [5]:
recognizer=unrestricted_recognizer(hmms=hmmss)


AttributeError: 'HMMState' object has no attribute 'parents'

In [ ]:
print_transition_information(recognizer,1)

Transit From <single_HMM.HMMState object at 0x00000293753E8650>
Transit to {<single_HMM.HMMState object at 0x00000293753E8650>: 0.9393939393939394, <single_HMM.HMMState object at 0x00000293753E8E90>: 0.06060606060606061}
The State Label is:0-1


In [ ]:
import numpy as np

# Define the dimensions of the array
n = 3  # Number of rows
m = 4  # Number of columns

# Initialize the 2D array with empty lists
empty_array = np.empty((n, m), dtype=object)

# Print the initialized array

empty_array


array([[None, None, None, None],
       [None, None, None, None],
       [None, None, None, None]], dtype=object)

In [ ]:
l=np.array([[1,4,2],[3,5,6]])
print(l)
print(l[0][1])



[[1 4 2]
 [3 5 6]]
4
